In [ ]:
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# LAB MODULE 4.
# Analysis of climate data
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

# Topics covered:
# (1) Climate anomalies
# (2) Climate indices
# (3) Empirical Orthogonal Functions

# We will work with datasets from CMIP6 archive; in the examples I will use one of the following cases:
#  psl_Amon_MRI-ESM2-0_historical_r1i1p1f1_gn_185001-201412.nc
#  psl_Amon_IPSL-CM6A-LR_historical_r1i1p1f1_gr_185001-201412.nc
#  psl_Amon_MIROC6_historical_r1i1p1f1_gn_185001-194912.nc

# You can download them either from the link provided in the slides, 
# or from the shared google drive link (login with your UNIMIB credentials only):
#  https://esgf-node.llnl.gov/search/cmip6/
#  https://drive.google.com/file/d/1-SOVeb7EoB_nDWcpugyNGTB6RHiRlRnF/view?usp=sharing

In [ ]:
# Libraries

import matplotlib.pyplot as plt
import matplotlib as mpl

import numpy as np

import cartopy.crs as ccrs

import xarray as xr

In [ ]:
# Example 1
# Calculate and display the EOF-NAO index from one of the CMIP6 historical simulations

In [ ]:
# Chose models

model = 'MRI-ESM2-0'

In [ ]:
#-------------------------------------------------
#-- Read and organize Sea Level Pressure data ----
#-------------------------------------------------

print('\n **** NOW ARRANGE SLP *** \n')

#- Path to file
modfile0=''.join(['./psl_Amon_',model,'_historical_r1i1p1f1_gn_185001-201412.nc'])
                   #psl_Amon_MRI-ESM2-0_historical_r1i1p1f1_gn_185001-201412.nc
#- explore file
ds0 = xr.open_dataset(modfile0)

#- extract variable (psl) - temporary variable
psl0 = ds0.psl
print(ds0)

In [ ]:
#- make some visual checks of what's going on
fig = plt.figure(figsize=(16,12))
subplots = (2,2)
n_panels = subplots[0] * subplots[1]

#- check: location before lon adjustment
ax = fig.add_subplot(subplots[0], subplots[1], 1, projection=ccrs.PlateCarree())
ax.set_global()
ax.coastlines() 
psl0m = psl0.mean(axis=0)
psl0m.plot(extend='both')

In [ ]:
# ~~~ Remember ~~~
# SLP anomalies over the Atlantic sector, 20°-80°N, 90°W-40°E. 

#- focus on the region of interest 
ds0_nat = ds0.sel(lat=slice(20,80), lon=slice(-90,40))
psl_nat=ds0_nat.psl

#- check
fig = plt.figure(figsize=(9,6))  # x,y(inches)

ax = plt.axes(projection=ccrs.PlateCarree())
ax.set_title(''.join([model,'\n']))
ax.set_global()
mm = ax.contourf(psl_nat.lon, psl_nat.lat, psl_nat.mean(axis=0), \
                   transform=ccrs.PlateCarree(),cmap=mpl.cm.jet )   

ax.coastlines()
ax.gridlines(draw_labels=True)

#- add colorbar
cbar_ax = fig.add_axes([0.28, 0.10, 0.46, 0.05]) #[left, bottom, width, height]
cbar = fig.colorbar(mm, cax=cbar_ax, extend='both', orientation='horizontal')
cbar.set_label('Pa')
cbar.ax.tick_params(labelsize=8)

plt.show()
plt.close()

In [ ]:
#- Adjust longitudes

#- (Option 1: in python)
ds0.coords['lon'] = (ds0.coords['lon'] + 180) % 360 - 180
ds0 = ds0.sortby(ds0.lon)

#- (Option 2: with nco from shell)
# ncap2 -O -s 'where(lon>180) lon=lon-360' ifile ofile

In [ ]:
# re-try after fixing longitudes

#- focus on the region of interest 
ds0_nat = ds0.sel(lat=slice(20,80), lon=slice(-90,40))
psl_nat=ds0_nat.psl

#- check
fig = plt.figure(figsize=(9,6))  # x,y(inches)

ax = plt.axes(projection=ccrs.PlateCarree())
ax.set_title(''.join([model,'\n']))
ax.set_global()
mm = ax.contourf(psl_nat.lon, psl_nat.lat, psl_nat.mean(axis=0), \
                   transform=ccrs.PlateCarree(),cmap=mpl.cm.jet )   

ax.coastlines()
ax.gridlines(draw_labels=True)

#- add colorbar
cbar_ax = fig.add_axes([0.28, 0.10, 0.46, 0.05]) #[left, bottom, width, height]
cbar = fig.colorbar(mm, cax=cbar_ax, extend='both', orientation='horizontal')
cbar.set_label('Pa')
cbar.ax.tick_params(labelsize=8)

plt.show()
plt.close()

In [ ]:
#- focus on the season of interest (boreal winter)

def djfm(month):
    return ((month == 1) | (month == 2) | (month == 3) | (month == 12))

psl_nat_m = psl_nat.sel(time=djfm(psl_nat['time.month']))
psl_nat_ms = psl_nat_m.coarsen(time=4).mean()

In [ ]:
#-- Calculate and plot NAO index based on EOFs ---

# Compute anomalies by removing the time-mean.
psl_anom = psl_nat_ms - psl_nat_ms.mean(dim='time')

from eofs.xarray import Eof

# Retrieve the leading EOF, expressed as the covariance (alternatives: e.g. correlation) 
# between the leading PC time series and the input SST anomalies at each grid point, and the
# leading PC time series itself.

solver = Eof(psl_anom)
eof1 = solver.eofsAsCovariance(neofs=1)
pc1 = solver.pcs(npcs=1, pcscaling=1)

In [ ]:
#- set multi-panel plot
fig = plt.figure(figsize=(8,12))
subplots = (2,1)
n_panels = subplots[0] * subplots[1]

#- plot leading EOF: spatial component of the NAO index
ax = fig.add_subplot(subplots[0], subplots[1], 1, projection=ccrs.PlateCarree())
ax.set_global()
ax.coastlines() 
eof1[0,:,:].plot()

#- plot leading EOF time series: temporal component of the NAO index
ax = fig.add_subplot(subplots[0], subplots[1], 2)
pc1.plot(marker='.')

#- Store/visualize plot
plt.show()
plt.close()

In [ ]:
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# Task 2. 
# Calculate the Southern Oscillation Index at monthly resolution 
# for two models of your choice for the period 1950-1999; 
# plot time series
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

